# Coming from Geopandas

If you're already using [geopandas.GeoDataFrame.explore](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html) to visualize STAC items, this notebook highlights some of the advantages of [stacmap.explore](https://stacmap.readthedocs.io/en/latest/api_reference.html) and some tips for transitioning.

Before we start, install the dependencies needed for `geopandas.GeoDataFrame.explore`:

In [ ]:
!pip install geopandas matplotlib mapclassify

Now the advantages of `stacmap`:

- Lightweight
  - `stacmap` only has a few dependencies to keep installs quick and easy.
- Native STAC support
  - `stacmap.explore` directly plots STAC items and collections without converting to GeoDataFrame first.
- Convenient features
  - Plot search extents, preview thumbnails, filter STAC properties by extension, etc.

## Native STAC Support

Exploring a collection of STAC items starts with loading items from a STAC, regardless of whether you use `geopandas` or `stacmap`.

In [1]:
import pystac
import stacmap
import geopandas as gpd

catalog = pystac.Catalog.from_file("https://planet.com/data/stac/disasters/hurricane-harvey/hurricane-harvey-0831/catalog.json")
print(catalog.description)

Planet Scenes and Composites for Hurricane Harvey on Aug 31, 2017


With `geopandas`, you'll need to manually convert that catalog into a `GeoDataFrame` before exploring it.

In [2]:
gdf = gpd.GeoDataFrame.from_features([item.to_dict() for item in catalog.get_all_items()], crs="EPSG:4326")
gdf.explore(column="gsd", categorical=True, cmap="Dark2")

`stacmap` offers native support for most STAC data types, so there's no need to build a GeoDataFrame. Just pass the STAC catalog directly into `explore`.

In [3]:
stacmap.explore(catalog, prop="gsd", cmap="Dark2", categorical=True)

## Convenient Features

`stacmap` also offers some features that you might frequently want when exploring STAC metadata.

### Search Extent

When searching for images, you might want to know how much of their footprint covers your area of interest. Pass a bounding box to `bbox` or a GeoJSON geometry to `intersects` to plot the search extent as an additional layer with `stacmap`.

In [4]:
bbox = [-96.3, 28.8, -93.5, 30.4]
stacmap.explore(catalog, prop="gsd", cmap="Dark2", categorical=True, bbox=bbox)

### Thumbnails

Some STAC items contain thumbnail assets. Preview the assets in their approximate location with `thumbnails=True`.

In [7]:
stacmap.explore(catalog, thumbnails=True, style_kwds=dict(fill=False))

### Metadata

`stacmap.explore` has some features to make metadata more useful:

- Metadata is sorted by prefix, with base properties first and extensions after. 
- The STAC ID of each item is added to the metadata table by default (this can be disabled with `add_id=False`).
- Display only relevant STAC extension properties by specifying a list of extensions (e.g. `extensions=["eo", "view"]`)
- Only include specific properties by specifying a list of properties (e.g. `props=["eo:cloud_cover"]`)

## Parameters

`geopandas.GeoDataFrame.explore` and `stacmap.explore` both have a *lot* of parameters. `stacmap` uses similar parameters whenever possible to make transitioning from `geopandas` easier, but some are renamed for clarity or dropped for simplicity, and others are added for additional functionality.

The lists below show how parameters differ between the two APIs.

| Changed            | Shared                          | Geopandas only        | stacmap only    |
|--------------------|---------------------------------|-----------------------|-----------------|
| `column` -> `prop` | `cmap`                          | `scheme`              | `name`          |
|                    | `m`                             | `k`                   | `bbox`          |
|                    | `tiles` & `attr`                | `categories`          | `intersects`    |
|                    | `tooltip` & `popup`             | `classification_kwds` | `thumbnails`    |
|                    | `highlight`                     | `marker_type`         | `fields`        |
|                    | `categorical`                   | `marker_kwds`         | `extensions`    |
|                    | `legend`                        | `legend_kwds`         | `shared_fields` |
|                    | `control_scale`                 |                       | `add_id`        |
|                    | `vmin` & `vmax`                 |                       | `zoom_to`       |
|                    | `width` & `height`              |                       | `layer_control` |
|                    | `style_kwds` & `highlight_kwds` |                       | `bounds_kwds`   |
|                    | `tooltip_kwds` & `popup_kwds`   |                       |                 |
|                    | `map_kwds`                      |                       |                 |